In [ ]:
# # Generate correlated Brownian motions
# @njit
# def generate_correlated_BMs(N, M, rho):
#     Z1 = np.random.randn(N, M)
#     Z2 = np.random.randn(N, M)
#     W1 = Z1
#     W2 = rho * Z1 + np.sqrt(1 - rho ** 2) * Z2
#     return W1, W2

# # Simulate Heston model paths
# @njit
# def simulate_heston_paths(S0, v0, kappa, theta, sigma, r, rho, T, M, N):
#     dt = T / M
#     S = np.zeros((N, M + 1))
#     v = np.zeros((N, M + 1))
#     S[:, 0] = S0
#     v[:, 0] = v0
#     W1, W2 = generate_correlated_BMs(N, M, rho)

#     for t in range(1, M + 1):
#         v[:, t] = np.maximum(
#             v[:, t-1] + kappa * (theta - v[:, t-1]) * dt + sigma * np.sqrt(v[:, t-1] * dt) * W2[:, t-1],
#             0
#         )
#         S[:, t] = S[:, t-1] * np.exp(
#             (r - 0.5 * v[:, t-1]) * dt + np.sqrt(v[:, t-1] * dt) * W1[:, t-1]
#         )
#     return S, v

# # Simulate Black-Scholes paths
# @njit
# def simulate_bs_paths(S0, sigma, r, T, M, N):
#     dt = T / M
#     S = np.zeros((N, M + 1))
#     S[:, 0] = S0
#     for t in range(1, M + 1):
#         Z = np.random.randn(N)
#         S[:, t] = S[:, t-1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z)
#     return S

# # Least Squares Monte Carlo pricing
# def least_squares_monte_carlo(S, K, r, T, M):
#     dt = T / M
#     intrinsic_values = np.maximum(K - S, 0)
#     cashflows = intrinsic_values[:, -1].copy()

#     for t in range(M - 1, 0, -1):
#         in_the_money = intrinsic_values[:, t] > 0
#         X = S[in_the_money, t].reshape(-1, 1)
#         Y = cashflows[in_the_money] * np.exp(-r * dt)
#         if len(Y) > 0:
#             model = LinearRegression().fit(X, Y)
#             continuation_value = model.predict(X)
#             exercise = intrinsic_values[in_the_money, t] > continuation_value
#             cashflows[in_the_money] = np.where(
#                 exercise,
#                 intrinsic_values[in_the_money, t],
#                 cashflows[in_the_money] * np.exp(-r * dt)
#             )
#     return np.mean(cashflows) * np.exp(-r * dt)

# # Batch simulation and pricing
# def batch_lsm_price(model, batch_size, num_batches):
#     prices = np.zeros(num_batches)
#     for i in range(num_batches):
#         if model == "heston":
#             S_batch, _ = simulate_heston_paths(S0, v0, kappa, theta, sigma, r, rho, T, M, batch_size)
#         elif model == "bs":
#             S_batch = simulate_bs_paths(S0, np.sqrt(theta), r, T, M, batch_size)
#         else:
#             raise ValueError("Model must be 'heston' or 'bs'")
#         prices[i] = least_squares_monte_carlo(S_batch, K, r, T, M)
#     return np.mean(prices)

# # Parameters
# np.random.seed(0)
# S0 = 100
# K = 100
# T = 1
# r = 0.05
# kappa = 2.0
# theta = 0.04
# sigma = 0.2
# rho = -0.7
# v0 = 0.04
# M = 50
# batch_size = 5000
# num_batches = 200

# Run simulations
# price_heston = batch_lsm_price("heston", batch_size, num_batches)
# price_bs = batch_lsm_price("bs", batch_size, num_batches)

# # Results
# df_results = pd.DataFrame({
#     "Model": ["Heston-LSM", "BS Option Price"],
#     "Price": [price_heston, price_bs]
# })
# print(df_results)